In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage import img_as_ubyte
import pickle

In [0]:
#4/zQFDcqJxIv-ozKrojm6LD8PrmNVQi31nQcpfDqU51i3XWOlyL9wh1ZU
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("/content/drive/My Drive/text-to-image")
os.listdir()

['model',
 'Flickr8k_Dataset',
 'data',
 'Flickr8k_text',
 'src-juan',
 'src-paola',
 'src-ely',
 'resultados_pasados',
 'pickles',
 'logs',
 'src-clean',
 'ToDo.gdoc']

## Armar con Word to Vec

In [0]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
.jog

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# GLOBALS
TOKEN_DIR = "./Flickr8k_text/"


In [0]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase 

In [0]:
# Función auxiliar para remover stopwords y otras normalizaciones de texto
def clean_sentences(dataframe, stopwords_extra,STOPWORDS ):
    table = str.maketrans('','',string.punctuation)
    SYNS = []
    for sentence in dataframe:
        sentence = re.sub(r"http\S+", "", sentence)
        sentence = decontracted(sentence)
        sentence = re.sub("\S*\d\S*", "", sentence).strip()
        sentence = re.sub('[^A-Za-z]+', ' ', sentence)
        sentence = ' '.join(e.translate(table) for e in sentence.split())
        sentence = ' '.join(e for e in sentence.split() if(len(e)>1) )
        sentence = ' '.join(e for e in sentence.split() if(e.isalpha()) )
        sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords_extra)
        sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in STOPWORDS)
        #sentence = stemmer.stem(sentence)
        SYNS.append(sentence.strip())
    return SYNS

In [0]:
# Natural language toolkit, nos ayudara con preprocessing de los datos
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# Imprime el set
print(STOPWORDS)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'few', "that'll", 'through', 'our', 'mustn', 'has', 'won', "you'll", 'd', "you'd", 'for', 'over', 'doesn', 'where', 'weren', 'same', "should've", 'herself', 'are', 'mightn', 'doing', 'were', 'so', "it's", 'each', "shouldn't", 'the', 'who', "hasn't", 'down', 'y', 'he', 'them', "won't", "wouldn't", "you've", 'isn', 'they', 'i', 'itself', 'an', 'up', 'too', 'is', 'couldn', 'themselves', 'between', 'am', 'further', 'no', 'from', 're', 'yours', 'having', 'needn', 'didn', 'had', 't', 'those', "don't", "haven't", 'whom', 'hers', 'being', 'before', 'o', "hadn't", 'ourselves', 'once', 'that', 'off', 'now', 'of', 'until', 'because', 'how', 'his', 'we', "weren't", 'during', 'all', 'at', 'other', 'any', 'll', 'such', 'some', "aren't", 'will', 'shouldn', 'than', 'himself', 'just', 'she', "she's", 'my', "you're", 'ma', 'by', 'what', 'while', 'then', 'more', 'hadn', 'to', 'out', 'hasn', 's'

In [0]:
# Set adicional de stopwords, removeremos palabras tanto de este set como del que
# viene definido en el nltk
stopwords_extra= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [0]:
import pandas as pd
TOKEN_DIR = "./Flickr8k_text/"
# Load data
dir_token = TOKEN_DIR + "Flickr8k.lemma.token.txt"
df_lemma = pd.read_csv(dir_token, sep='\t', names=['ID', 'Text'], index_col=False)
print(df_lemma.shape)

# Remueve entries with nan values
df_lemma.dropna(inplace=True)
df_lemma.isnull().values.any()
df_lemma.columns = ['ID', 'lemma']
df_lemma

import pandas as pd
TOKEN_DIR = "./Flickr8k_text/"
# Load data
dir_token = TOKEN_DIR + "Flickr8k.token.txt"
df_token = pd.read_csv(dir_token, sep='\t', names=['ID', 'Text'], index_col=False)
print(df_token.shape)

# Remueve entries with nan values
df_token.dropna(inplace=True)
df_token.isnull().values.any()
df_token.head()

df_token.columns = ['ID', 'token']
df_token

df = df_token.merge(df_lemma, how='inner')
print(df.shape)
from sklearn.model_selection import train_test_split

X = df.lemma
#X_lemma = df.lemma
y = df.ID

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#X_lemma_train, X_lemma_test, y_train, y_test = train_test_split(X_lemma, y, test_size=0.20, random_state=42)

(40460, 2)
(40460, 2)
(40460, 3)


In [0]:
df_y_test = pd.DataFrame(y_test)
df_X_test = pd.DataFrame(X_test)

test_mask = (df_y_test.ID!='2258277193_586949ec62.jpg.1#0').tolist()
df_y_test = df_y_test[test_mask]
df_X_test = df_X_test[test_mask]

df_y_test_new = df_y_test.ID
df_X_test_new = df_X_test.lemma

df_y_train = pd.DataFrame(y_train)
df_X_train = pd.DataFrame(X_train)

print(len(df_y_train))
train_mask1 = (df_y_train.ID!='2258277193_586949ec62.jpg.1#2').tolist()
print(sum(train_mask1))
df_y_train = df_y_train[train_mask1]
df_X_train = df_X_train[train_mask1]
print(len(df_y_train))

train_mask1 = (df_y_train.ID!='2258277193_586949ec62.jpg.1#3' ).tolist()
df_y_train = df_y_train[train_mask1]
df_X_train = df_X_train[train_mask1]
print(len(df_y_train))

train_mask1 = (df_y_train.ID!='2258277193_586949ec62.jpg.1#4' ).tolist()
df_y_train = df_y_train[train_mask1]
df_X_train = df_X_train[train_mask1]
print(len(df_y_train))

train_mask1 = (df_y_train.ID!='2258277193_586949ec62.jpg.1#1' ).tolist()
df_y_train = df_y_train[train_mask1]
df_X_train = df_X_train[train_mask1]
print(len(df_y_train))

df_y_train_new = df_y_train.ID
df_X_train_new = df_X_train.lemma

#print(len(df_y_train_new))
#print(len(df_y_train))

32368
32367
32367
32366
32365
32364


In [0]:
print(len(df_y_train_new))
print(len(df_y_train))

32364
32364


In [0]:
# Remueve stopwords
synops_train = clean_sentences(df_X_train_new, stopwords_extra,STOPWORDS )
synops_test  = clean_sentences(df_X_test_new, stopwords_extra,STOPWORDS )

In [0]:
# Opcion A Tokenizer

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Entrena un Tokenizer
# Crea un diccionario numerado de las palabras existentes en el corpus, y devuelve
# por cada palabra, el número entero de su índice en el diccionario.
# En este caso, considera las 2000 palabras más frecuentes.
# oov_token = constante asiganda para palabras fuera del vocabulario
#5716
TAM_VOCAB = 5748
tokenizer = Tokenizer(num_words=TAM_VOCAB , oov_token='<OOV>')
tokenizer.fit_on_texts(synops_train)

print("Number of words in the dictionary", len(tokenizer.word_index))
print(list(tokenizer.word_index.items())[:10])

Number of words in the dictionary 5748
[('<OOV>', 1), ('dog', 2), ('man', 3), ('two', 4), ('boy', 5), ('girl', 6), ('woman', 7), ('white', 8), ('person', 9), ('black', 10)]


In [0]:
train_sequences[:2]

[[27, 6, 29, 17, 64], [4, 17, 35, 18, 3426]]

In [0]:
# Convierte palabras en enteros
train_sequences = tokenizer.texts_to_sequences(synops_train)
lengths = []
for seq in train_sequences:
    lengths.append(len(seq))

print("Shortest article with", np.array(lengths).min(), "words.")
print("Longest article with", np.array(lengths).max(), "words.")
print(f"Average length: {np.array(lengths).mean():.2f} +- {np.array(lengths).std():.2f} words.")
print('\n')

for seq in train_sequences[:10]:
    print(seq)

Shortest article with 0 words.
Longest article with 20 words.
Average length: 6.21 +- 2.07 words.


[3, 52, 2, 14, 901, 63, 293]
[84, 29, 147, 16]
[38, 9, 515, 277, 2297, 115, 2297, 1048]
[7, 17, 663, 120, 1198, 391, 552, 111]
[32, 27, 6, 47, 8, 60, 10, 167, 976]
[19, 18, 409, 622, 41, 37, 111]
[6, 128, 552, 80]
[4, 5, 163, 96]
[3, 162, 28, 265]
[10, 2, 76, 368, 78, 43]


In [0]:
# Hace todos los artículos del mismo tamaño usando padding y truncando
# Usaré maxlen = 500 (aprox) average length
MAX_LEN = 21
x_train = pad_sequences(train_sequences, maxlen=MAX_LEN , padding='pre', truncating='pre')
lengths = []
for seq in x_train:
    lengths.append(len(seq))

print("Shortest article with", np.array(lengths).min(), "words.")
print("Longest article with", np.array(lengths).max(), "words.")
print("Average length:", np.array(lengths).mean(), "words.")
print('\n')

print((train_sequences[0]))
print((x_train[0]))

print((train_sequences[1]))
print((x_train[1]))

print((train_sequences[10]))
print((x_train[10]))

print('\n Max word index:', x_train.max())

Shortest article with 21 words.
Longest article with 21 words.
Average length: 21.0 words.


[3, 52, 2, 14, 901, 63, 293]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   3  52   2  14
 901  63 293]
[84, 29, 147, 16]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  84
  29 147  16]
[6, 8, 67, 719]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   6
   8  67 719]

 Max word index: 5747


In [0]:
test_sequences = tokenizer.texts_to_sequences(synops_test)
x_test = pad_sequences(test_sequences, maxlen=MAX_LEN , padding='pre', truncating='post')

In [0]:
normalized_train = x_train/5716
normalized_test = x_test/5716

In [0]:
synops_train

In [0]:
from pickle import dump, load

with open("./pickles/simple/emb_train_norm.pickle","wb") as f:
  pickle.dump(normalized_train, f)

with open("./pickles/simple/emb_test_norm.pickle","wb") as f:
  pickle.dump(normalized_test, f)

with open("./pickles/simple/emb_train.pickle","wb") as f:
  pickle.dump(x_train, f)

with open("./pickles/simple/emb_test.pickle","wb") as f:
  pickle.dump(x_test, f)

with open("./pickles/simple/x_train_name.pickle","wb") as f:
  pickle.dump(df_y_train_new, f)

with open("./pickles/simple/x_test_name.pickle","wb") as f:
  pickle.dump(df_y_test_new, f)

with open("./pickles/simple/lemma_train.pickle","wb") as f:
  pickle.dump(synops_train, f)

with open("./pickles/simple/lemma_test.pickle","wb") as f:
  pickle.dump(synops_test, f)


In [0]:
x_train.shape

(32366, 21)

In [0]:
# One hot encoding multiple
MAX_WORD = 5716
NUM_VECTOR = x_test.shape[1]
NUM_VECTORES = x_test.shape[0]

TEST_ONE_HOT = []

for j in range(NUM_VECTORES):
  one_hot = [0]*MAX_WORD
  for i in range(NUM_VECTOR):
    one_hot[x_test[j][i]-1] = 1
  TEST_ONE_HOT.append(one_hot)


MAX_WORD = 5716
NUM_VECTOR = x_train.shape[1]
NUM_VECTORES = x_train.shape[0]

TRAIN_ONE_HOT = []

for j in range(NUM_VECTORES):
  one_hot = [0]*MAX_WORD
  for i in range(NUM_VECTOR):
    one_hot[x_train[j][i]-1] = 1
  TRAIN_ONE_HOT.append(one_hot) 


with open("./pickles/simple/one_hot_train.pickle","wb") as f:
  pickle.dump(TRAIN_ONE_HOT, f)

with open("./pickles/simple/one_hot_test.pickle","wb") as f:
  pickle.dump(TEST_ONE_HOT, f)

In [0]:
len(TRAIN_ONE_HOT)

32366

In [0]:
NUM_VECTORES

32366

In [0]:
sum(one_hot)

5

In [0]:
# One hot encoding multiple
x_test.shape
MAX_WORD =8
NUM_VECTOR = x_test.shape[1]

A = [1,5,6,7]
one_hot = [0]*MAX_WORD
for i in range(4):
  one_hot[A[i]-1] = 1

one_hot

[1, 0, 0, 0, 1, 1, 1, 0]

In [0]:
df_X_train_new

32128             A little girl ride in a child 's swing .
38063                  two child in a field of red poppies
29827        a man wear a cowboy hat hold a protest sign .
6199     A car leave a dust trail as it go around the d...
25282    two man on bicycle on the side of a snowpacked...
                               ...                        
6265                                      A boy skateboard
11284       A brown and white dog be run through a field .
38158                 two player run for the soccer ball .
860                    A boy skateboard on a cement wall .
15795    A dog be play by itself on a beach that have r...
Name: Text, Length: 32366, dtype: object

# Con Word2Vec

In [0]:
# Remueve stopwords
# synops_train 

In [0]:
from gensim.models import Word2Vec

In [0]:
import re

sentences_list = []

for sentence in synops_train:
  wordList = re.sub("[^\w]", " ",  sentence).split()
  sentences_list.append(wordList)

In [0]:
w2v_model = Word2Vec(min_count=1,
                     window=3,
                     size=64,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=-1)

In [0]:
from time import time
t = time()
w2v_model.build_vocab(sentences_list , progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.02 mins


In [0]:
w2v_model.train(sentences_list, total_examples=w2v_model.corpus_count, epochs=1000, report_delay=1)

(0, 0)

In [0]:
w2v_model.wv.most_similar(positive=["yellow"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bustle', 0.4620729088783264),
 ('attend', 0.44039782881736755),
 ('rag', 0.43986040353775024),
 ('secured', 0.42919832468032837),
 ('slighty', 0.4237280786037445),
 ('memorabilia', 0.41043317317962646),
 ('ashaft', 0.3829176425933838),
 ('bug', 0.38072019815444946),
 ('religious', 0.37624743580818176),
 ('nightclub', 0.3728558421134949)]

In [0]:
print(len(w2v_model['yellow']))

64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [0]:
model = Word2Vec(sentences_list)
words = list(model.wv.vocab)
print(words)


['yellow', 'dog', 'run', 'trail', 'woman', 'walk', 'sidewalk', 'past', 'halloween', 'people', 'outdoors', 'snowy', 'mountain', 'two', 'sumo', 'wrestler', 'fight', 'ring', 'hold', 'frisbee', 'boy', 'wear', 'jacket', 'biker', 'go', 'handlebar', 'fly', 'ground', 'man', 'stand', 'rocky', 'front', 'little', 'girl', 'pink', 'clothes', 'rod', 'wet', 'red', 'orange', 'ball', 'mouth', 'swim', 'water', 'purple', 'shirt', 'blue', 'short', 'play', 'parent', 'young', 'male', 'pose', 'picture', 'fountain', 'white', 'gold', 'black', 'hat', 'sit', 'cross', 'legged', 'subway', 'train', 'small', 'child', 'winter', 'smile', 'snow', 'away', 'outdoor', 'market', 'jump', 'across', 'sand', 'stick', 'huge', 'pile', 'around', 'flowery', 'dress', 'break', 'tree', 'stump', 'track', 'near', 'station', 'lined', 'building', 'group', 'middle', 'eastern', 'person', 'come', 'underwater', 'lit', 'scene', 'kiss', 'catch', 'park', 'kid', 'ride', 'bike', 'ocean', 'even', 'eat', 'bench', 'brown', 'leash', 'next', 'baseball

In [0]:
print(len(model['yellow']))

100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
